# Filtres convolutifs -- Pistes de solutions
Matériel de cours rédigé par Pascal Germain, 2019
************

### Changer la taille et le nombre de filtres convolutifs

Ici, nous réutilisons la classe `UneArchiPourMNIST` pour obtenir 64 filtres de taille 9 chacun.

In [ ]:
mon_archi = UneArchiPourMNIST(nb_filtres=64, taille_noyau=9)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)

### Ajouter une ou plusieurs couches de filtres convolutifs dans la première partie du réseau

Modifions la classe `UneArchiPourMNIST` afin d'ajouter une 2e couche de filtres  convolutifs.

In [ ]:
class Archi2CouchesCovolutives(nn.Module):
    def __init__(self, couche1_nb_filtres=16, couche1_taille_noyau=9, 
                       couche2_nb_filtres=32, couche2_taille_noyau=3):
        # Initialisation de la classe de base nn.Module
        super().__init__()
        
        # Créons deux couches de convolution 
        self.modele_conv = nn.Sequential(
            # Première couche de convolution
            nn.Conv2d(1, couche1_nb_filtres, kernel_size=couche1_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # NOUVEAUTÉ: Deuxième couche de convolution
            nn.Conv2d(couche1_nb_filtres, couche2_nb_filtres, kernel_size=couche2_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche de sortie
        # NOUVEAUTÉ: Calcul du nombre de neurones sur la couche pleinement connectée
        #            considérant la 2e couche de convolution
        nb_pixels = (28 - couche1_taille_noyau + 1) // 2 
        nb_pixels = (nb_pixels - couche2_taille_noyau + 1) // 2
        self.nb_neurones_du_milieu = couche2_nb_filtres * (nb_pixels**2)
        
        self.modele_plein = nn.Sequential(
            nn.Linear(self.nb_neurones_du_milieu, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3

In [ ]:
mon_archi = Archi2CouchesCovolutives(couche1_nb_filtres=16, couche1_taille_noyau=9,
                                     couche2_nb_filtres=32, couche2_taille_noyau=3)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)

### Ajouter davantage de couches pleinement connectées dans la seconde partie du réseau

Reprenons l'architecture `Archi2CouchesCovolutives` ci-haut et ajoutons-y une couche cachée pleinement connectée

In [ ]:
class Archi2CouchesCovolutives2CouchesPC(nn.Module):
    def __init__(self, couche1_nb_filtres=16, couche1_taille_noyau=9, 
                       couche2_nb_filtres=32, couche2_taille_noyau=3,
                       couche3_nb_neurones=100):
        # Initialisation de la classe de base nn.Module
        super().__init__()
        
        # Créons deux couches de convolution 
        self.modele_conv = nn.Sequential(
            # Première couche de convolution
            nn.Conv2d(1, couche1_nb_filtres, kernel_size=couche1_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # Deuxième couche de convolution
            nn.Conv2d(couche1_nb_filtres, couche2_nb_filtres, kernel_size=couche2_taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche cachée pleinement connectée
        nb_pixels = (28 - couche1_taille_noyau + 1) // 2
        nb_pixels = (nb_pixels - couche2_taille_noyau + 1) // 2
        self.nb_neurones_du_milieu = couche2_nb_filtres * (nb_pixels**2)
        
        self.modele_plein = nn.Sequential(
            # NOUVEAUTÉ: Nouvelle couche cachée pleinement connectée avec activation ReLU
            nn.Linear(self.nb_neurones_du_milieu, couche3_nb_neurones),  
            nn.ReLU(),
            # Couche de sortie
            nn.Linear(couche3_nb_neurones, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):         
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3

In [ ]:
mon_archi = Archi2CouchesCovolutives2CouchesPC(couche1_nb_filtres=16, couche1_taille_noyau=9,
                                               couche2_nb_filtres=32, couche2_taille_noyau=3,
                                               couche3_nb_neurones=100)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)